In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
ratings = pd.read_csv('ratings_Electronics.csv', names=['user','asin','rating','timestamp'])

In [4]:
ratings.shape

(7824482, 4)

In [5]:
ratings.head()

,user,asin,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [6]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

metadata_df = getDF('meta_Electronics.json.gz')

In [7]:
metadata_df.shape

(498196, 9)

In [8]:
metadata_df.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [9]:
#Merge the two dataframes above to create input dataframe for recommender systems
Electronics = pd.merge(ratings, metadata_df, on="asin", how="left") 

In [10]:
Electronics.shape

(7824482, 12)

In [11]:
Electronics.columns

Index(['user', 'asin', 'rating', 'timestamp', 'imUrl', 'description',
       'categories', 'title', 'price', 'salesRank', 'related', 'brand'],
      dtype='object')

In [12]:
Electronics_df= Electronics[['asin','title','user','rating']]

In [13]:
Electronics_df.shape

(7824482, 4)

In [14]:
Electronics_df.columns

Index(['asin', 'title', 'user', 'rating'], dtype='object')

In [15]:
Electronics_df = Electronics_df.rename(columns={'asin': 'prod_id', 'title': 'prod_name', 'user': 'user_id'})

In [16]:
Electronics_df.columns

Index(['prod_id', 'prod_name', 'user_id', 'rating'], dtype='object')

In [17]:
Electronics_df.head()

,prod_id,prod_name,user_id,rating
0,0132793040,Kelby Training DVD: Mastering Blend Modes in A...,AKM1MP6P0OYPR,5.0
1,0321732944,Kelby Training DVD: Adobe Photoshop CS5 Crash ...,A2CX7LUOHB2NDG,5.0
2,0439886341,Digital Organizer and Messenger,A2NWSAGRHCP8N5,1.0
3,0439886341,Digital Organizer and Messenger,A2WNBOD3WNDNKT,3.0
4,0439886341,Digital Organizer and Messenger,A1GI0U4ZRJA8WN,1.0


In [18]:
Electronics_grouped = Electronics_df.groupby(['user_id'],as_index=False)['rating'].sum()

In [19]:
ratings_50=Electronics_grouped[(Electronics_grouped['rating']>=50)]

In [20]:
ratings_50.shape

(41527, 2)

In [21]:
ratings_50.columns

Index(['user_id', 'rating'], dtype='object')

In [22]:
ratings_50.head()

,user_id,rating
1175,A0251761JI35FM4C8VK2,55.0
1247,A02712303HM5RXRLNJUB7,53.0
1288,A0279100VZXR9A2495P4,60.0
1315,A0284208PB0CNSHI1OC6,84.0
1377,A02970121VCH64N53W9F4,52.0


In [23]:
Electronics_50 = Electronics_df[Electronics_df['user_id'].isin(ratings_50['user_id'])]

In [24]:
Electronics_50.shape

(853459, 4)

In [25]:
Electronics_50.head()

,prod_id,prod_name,user_id,rating
6,0511189877,CLIKR-5 Time Warner Cable Remote Control UR5U-...,A3J3BRHTDRFJ2G,2.0
16,0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,A3N7T0DY83Y4IG,3.0
17,0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,A1H8PY3QHMQQA0,2.0
89,0594451647,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,AAZ084UMH8VZ2,5.0
94,0594451647,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,A3BY5KCNQZXV5U,5.0


In [26]:
Electronics_50.dtypes

prod_id       object
prod_name     object
user_id       object
rating       float64
dtype: object

In [27]:
Electronics_50['rating'].isna().sum()

0

In [28]:
%matplotlib inline

import pandas
from sklearn.cross_validation import train_test_split
import numpy as np
import time
from sklearn.externals import joblib
import Recommenders as Recommenders
import Evaluation as Evaluation

C:\Users\aniru\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [29]:
train_data, test_data = train_test_split(Electronics_50, test_size = 0.30, random_state=0)
train_data.head(5)

,prod_id,prod_name,user_id,rating
3899909,B004616OIQ,Aurum Ultra Series - High Speed HDMI Cable wit...,A297Y763KFR9Y7,2.0
6778124,B009YB00HO,BestDealUSA Pure Shallow Blue TPU Gel Rubber S...,A6EPTR9ABAQTU,5.0
5743480,B007GR44PM,Tap2Pass Receiver,A17V2N0N8CXSA9,5.0
6734277,B009USAJCC,Anker&reg; 2nd Gen Astro E3 10000mAh Dual-Port...,A1T1YSCDW0PD25,5.0
3916192,B0046TJG1U,IOGEAR SD/MicroSD/MMC Card Reader/Writer GFR20...,A1MD9AEU9C4WJN,4.0


## Popularity Recommender model

### Simple popularity-based recommender class (Can be used as a black box)

#### Create an instance of popularity based recommender class

In [30]:
pm = Recommenders.popularity_recommender_py()
pm.create(train_data, 'user_id', 'prod_name')

#### Use the popularity model to make some predictions

In [31]:
users=Electronics_50['user_id'].unique()

In [32]:
len(users)

41527

In [33]:
print(users)

['A3J3BRHTDRFJ2G' 'A3N7T0DY83Y4IG' 'A1H8PY3QHMQQA0' ... 'A2QRL5FSRPK4LM'
 'A2WYMB9ZYTOFR1' 'A1PB6OT0OOPKNQ']


In [34]:
user_id = users[1]
pm.recommend(user_id)

,user_id,prod_name,score,Rank
105271,A3N7T0DY83Y4IG,SanDisk Ultra 64GB MicroSDXC Class 10 UHS Memo...,1393,1.0
13163,A3N7T0DY83Y4IG,AmazonBasics High-Speed HDMI Cable - 15 Feet (...,1184,2.0
73841,A3N7T0DY83Y4IG,Mediabridge ULTRA Series HDMI Cable (6 Feet) -...,870,3.0
49799,A3N7T0DY83Y4IG,Google Chromecast HDMI Streaming Media Player,869,4.0
122075,A3N7T0DY83Y4IG,Transcend 8 GB Class 10 SDHC Flash Memory Card...,774,5.0
133833,A3N7T0DY83Y4IG,eneloop SEC-CSPACER4PK C Size Spacers for use ...,701,6.0
13135,A3N7T0DY83Y4IG,AmazonBasics Apple Certified Lightning to USB ...,689,7.0
35638,A3N7T0DY83Y4IG,DVI Gear HDMI Cable 2M 6 feet,689,8.0
104851,A3N7T0DY83Y4IG,SanDisk 4GB Extreme SDHC Class 10 Memory Card,634,9.0
89027,A3N7T0DY83Y4IG,"Panasonic RPHJE120D In-Ear Headphone, Orange",582,10.0


In [35]:
user_id = users[41526]
pm.recommend(user_id)

,user_id,prod_name,score,Rank
105271,A1PB6OT0OOPKNQ,SanDisk Ultra 64GB MicroSDXC Class 10 UHS Memo...,1393,1.0
13163,A1PB6OT0OOPKNQ,AmazonBasics High-Speed HDMI Cable - 15 Feet (...,1184,2.0
73841,A1PB6OT0OOPKNQ,Mediabridge ULTRA Series HDMI Cable (6 Feet) -...,870,3.0
49799,A1PB6OT0OOPKNQ,Google Chromecast HDMI Streaming Media Player,869,4.0
122075,A1PB6OT0OOPKNQ,Transcend 8 GB Class 10 SDHC Flash Memory Card...,774,5.0
133833,A1PB6OT0OOPKNQ,eneloop SEC-CSPACER4PK C Size Spacers for use ...,701,6.0
13135,A1PB6OT0OOPKNQ,AmazonBasics Apple Certified Lightning to USB ...,689,7.0
35638,A1PB6OT0OOPKNQ,DVI Gear HDMI Cable 2M 6 feet,689,8.0
104851,A1PB6OT0OOPKNQ,SanDisk 4GB Extreme SDHC Class 10 Memory Card,634,9.0
89027,A1PB6OT0OOPKNQ,"Panasonic RPHJE120D In-Ear Headphone, Orange",582,10.0


## Collaborative filtering model

### Matrix factorization via Singular value decomposition

In [36]:
prodid= Electronics_50.prod_id.unique()
np.count_nonzero(prodid)
np.sort(prodid)

array(['0511189877', '0528881469', '0594451647', ..., 'B00LPQRT34',
       'B00LXEC8CU', 'BT008UKTMW'], dtype=object)

###### Lets consider a small subset of the whole data, as it is taking longer time to run the model on complete data

In [37]:
train_df, test_df = train_test_split(Electronics_50, test_size = 0.99, random_state=0)
train_df.shape

(8534, 4)

In [38]:
Electronics_50.head()

,prod_id,prod_name,user_id,rating
6,0511189877,CLIKR-5 Time Warner Cable Remote Control UR5U-...,A3J3BRHTDRFJ2G,2.0
16,0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,A3N7T0DY83Y4IG,3.0
17,0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,A1H8PY3QHMQQA0,2.0
89,0594451647,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,AAZ084UMH8VZ2,5.0
94,0594451647,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,A3BY5KCNQZXV5U,5.0


In [39]:
# We want the format of ratings matrix to be one row per user and one column per movie. 
#we can pivot ratings_df to get that and call the new variable R_df.
R_df = train_df.pivot(index = 'user_id', columns ='prod_id', values = 'rating').fillna(0)

In [40]:
R_df.shape

(7423, 6977)

In [41]:
R_df.head().T

user_id,A100WO06OQR8BQ,A1031R8HD3E4GL,A103B6MQ5IF2BK,A103QH6R6A4FT1,A1047HEENF4C0C
prod_id,,,,,
1400501466,0.0,0.0,0.0,0.0,0.0
1400532655,0.0,0.0,0.0,0.0,0.0
140053271X,0.0,0.0,0.0,0.0,0.0
9989499608,0.0,0.0,0.0,0.0,0.0
B00000J05A,0.0,0.0,0.0,0.0,0.0
B00000J0IX,0.0,0.0,0.0,0.0,0.0
B00000J1E6,0.0,0.0,0.0,0.0,0.0
B00000J1SC,0.0,0.0,0.0,0.0,0.0
B00000J1TS,0.0,0.0,0.0,0.0,0.0


In [42]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_df, k = 50)

In [43]:
sigma = np.diag(sigma)

In [44]:
#I also need to add the user means back to get the predicted 5-star ratings
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [45]:
preds_df.set_index(R_df.index, inplace=True)

In [46]:
preds_df.head()

prod_id,1400501466,1400532655,140053271X,9989499608,B00000J05A,B00000J0IX,B00000J1E6,B00000J1SC,B00000J1TS,B00000J434,...,B00K806YFM,B00K8GSH8S,B00K91DB7Y,B00KFAGCUM,B00KH8PUDW,B00KL5NWHM,B00KOHQU58,B00KQCJQKW,B00KR6FY94,B00KSBB84S
user_id,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,-1.510700e-33,2.464614e-37,-5.481067e-34,9.994358e-34,-5.499522e-35,-2.984764e-34,-9.997364e-34,7.599240e-20,-8.084748e-35,3.766014e-34,...,7.671597e-35,-2.434056e-34,-1.746920e-34,-1.885612e-34,7.686789e-34,-3.132214e-34,-2.801512e-34,-7.793003e-34,1.926217e-34,9.536013e-36
A1031R8HD3E4GL,-8.570266e-34,-9.326432e-37,-3.889448e-33,8.095123e-34,-5.610049e-34,1.410046e-34,-7.931969e-34,-5.497638e-19,-1.408941e-33,1.029356e-32,...,-6.475740e-34,-2.295751e-33,-1.777913e-33,2.861063e-35,-1.360756e-32,5.115802e-34,-2.362004e-34,-5.990935e-34,2.781552e-33,-8.316900e-35
A103B6MQ5IF2BK,2.265943e-31,1.272578e-35,-8.044810e-33,-4.349055e-33,1.822978e-34,-5.247598e-33,4.195225e-33,1.422460e-18,-1.896813e-33,2.296229e-32,...,3.541864e-34,5.373066e-34,-1.767883e-32,-2.217041e-33,-9.228258e-32,-2.950831e-33,1.263561e-33,3.182405e-33,3.686011e-33,-6.729539e-35
A103QH6R6A4FT1,2.108903e-33,-4.872978e-37,9.410260e-34,-1.458456e-33,8.855864e-35,4.407481e-34,1.456893e-33,-8.465269e-20,1.133738e-34,-6.877813e-34,...,-1.996104e-34,3.917824e-34,2.074104e-34,2.893986e-34,-1.126584e-33,4.706079e-34,4.088436e-34,1.135933e-33,-2.793361e-34,-2.596101e-35
A1047HEENF4C0C,-3.966994e-17,-5.779320e-22,-4.007739e-19,8.517100e-19,2.997788e-20,1.333348e-18,-4.906899e-19,2.105776e-16,-8.368144e-19,1.345297e-17,...,1.618046e-18,-1.743518e-19,-5.303432e-18,1.109473e-18,1.600746e-17,1.553109e-18,-1.957213e-19,-6.042692e-19,1.271979e-18,2.374434e-19


In [47]:
preds_df.shape

(7423, 6977)

In [48]:
sigma

array([[11.41154728,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 11.44122806,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 11.45616411, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., 17.81624419,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        18.48646943,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 22.61780995]])

In [49]:
# return the products with the highest predicted rating that the specified user hasn’t already rated
#Take specific user row from matrix from predictions
def recommend_products(predictions_df, userID, products_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    #user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.loc[userID].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.user_id == (userID)]
    #Added title and genres
    user_full = (user_data.merge(products_df, how = 'left', left_on = 'prod_id', right_on = 'prod_id').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} products.'.format(userID, user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings products not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (products_df[~products_df['prod_id'].isin(user_full['prod_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'prod_id',
               right_on = 'prod_id').
         rename(columns = {userID: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations, sorted_user_predictions, user_data, user_full

In [50]:
prod_count=Electronics_df.groupby("prod_id", as_index=False).count()

In [51]:
Items_df=Electronics_df[Electronics_df['prod_id'].isin(prod_count['prod_id'])]

In [52]:
Items_df = Items_df[['prod_id']]

In [53]:
Items_df.head()

,prod_id
0,0132793040
1,0321732944
2,0439886341
3,0439886341
4,0439886341


In [54]:
already_rated, predictions, sorted_user_predictions, user_data, user_full = recommend_products(preds_df, 'A1031R8HD3E4GL', Items_df, Electronics_50, 10)

User A1031R8HD3E4GL has already rated 5806 products.
Recommending the highest 10 predicted ratings products not already rated.


In [55]:
already_rated

,prod_id,prod_name,user_id,rating
0,B00005B98K,Philips DVD712AT DVD Player,A1031R8HD3E4GL,5.0
3620,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3612,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3613,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3614,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3615,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3616,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3617,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3618,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3619,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0


In [56]:
predictions

,prod_id
6902255,B00AJHDZSI
6902639,B00AJHDZSI
6902646,B00AJHDZSI
6902645,B00AJHDZSI
6902644,B00AJHDZSI
6902643,B00AJHDZSI
6902642,B00AJHDZSI
6902641,B00AJHDZSI
6902640,B00AJHDZSI
6902638,B00AJHDZSI


In [57]:
user_data

,prod_id,prod_name,user_id,rating
96547,B00005B98K,Philips DVD712AT DVD Player,A1031R8HD3E4GL,5.0
407608,B0001U6RNI,Toshiba SD-3960 DVD Player,A1031R8HD3E4GL,5.0
452178,B0002C7YFK,Magnavox DVD Player (MDV45817) (MDV45817),A1031R8HD3E4GL,4.0
559604,B0006B486K,Koss KSC75 Portable Stereophone Headphones,A1031R8HD3E4GL,5.0
589092,B0007KQUP2,Canon PowerShot A510 3.2MP Digital Camera with...,A1031R8HD3E4GL,5.0
631197,B0007XJSQC,Sennheiser HD201 Lightweight Over-Ear Binaural...,A1031R8HD3E4GL,4.0
733939,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
1224943,B000NUYW92,JVC HARX300 Full-Size Headphones (Black),A1031R8HD3E4GL,3.0
1990104,B001F51G0W,Logitech Z-5 USB Stereo Speakers for Mac and PC,A1031R8HD3E4GL,4.0
2272242,B001S0I1M0,Sylvania NB530SLX Blu-ray Disc Player (Black),A1031R8HD3E4GL,5.0


In [58]:
user_full

,prod_id,prod_name,user_id,rating
0,B00005B98K,Philips DVD712AT DVD Player,A1031R8HD3E4GL,5.0
3620,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3612,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3613,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3614,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3615,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3616,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3617,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3618,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0
3619,B000B60H0G,Samsung BD-C5500 1080p Blu-ray Disc Player,A1031R8HD3E4GL,5.0


In [59]:
sorted_user_predictions

prod_id
B00AJHDZSI    5.842312e-17
B000Q8UAWY    4.777936e-17
B002JQNXZC    4.261940e-17
B0079UAT0A    3.788991e-17
B000QUUFRW    3.677962e-17
B0012S4APK    3.590076e-17
B00622AG6S    2.875499e-17
B001TH7T2U    2.565298e-17
B003LSTD38    2.503507e-17
B007R5YDYA    2.207356e-17
B003ES5ZUU    2.205467e-17
B0041Q38NU    2.186576e-17
B0019EHU8G    2.055968e-17
B002WE6D44    2.023921e-17
B00GMG2ENM    1.982703e-17
B008OHNZI0    1.917458e-17
B002YU83YO    1.894308e-17
B0002L5R78    1.822964e-17
B0088PUEPK    1.768910e-17
B004TJ6JH6    1.747458e-17
B000EITTLE    1.648719e-17
B005FYNSPK    1.504666e-17
B004YHXXKO    1.173198e-17
B003RRYAXQ    1.173198e-17
B002BH3Z8E    1.154037e-17
B005HSDLCO    1.117820e-17
B004FV67XE    1.098146e-17
B008A3KFB8    1.098146e-17
B007WTAJTO    1.034116e-17
B002J9HBIO    1.015935e-17
                  ...     
B002TLH4OI   -1.435573e-17
B00B8O6IIK   -1.435573e-17
B00HFFDDLG   -1.435573e-17
B005GTNZUM   -1.588260e-17
B00FB1NTP6   -1.601080e-17
B00BLLX9CI   -1.8366

### Build a product recommender with personalization

We now create an item similarity based collaborative filtering model that allows us to make personalized recommendations to each user.

#### Class for an item similarity based personalized recommender system
#### Create an instance of item similarity based recommender class

In [85]:
is_model = Recommenders.item_similarity_recommender_py()
is_model.create(train_data, 'user_id', 'prod_name')

In [ ]:
#Print the products for the user in training data
user_id = users[15]
user_items = is_model.get_user_items(user_id)
#
print("------------------------------------------------------------------------------------")
print("Training data products for the user userid: %s:" % user_id)
print("------------------------------------------------------------------------------------")

for user_item in user_items:
    print(user_item)

print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

#Recommend songs for the user using personalized model
is_model.recommend(user_id)

------------------------------------------------------------------------------------
Training data songs for the user userid: A3BMUBUC1N77U8:
------------------------------------------------------------------------------------
Metra 40-CR10 Chrysler 2002 Antenna Adapter Cable
AmazonBasics HDMI to DVI  Adapter Cable (9.8 Feet/3.0 Meters)
C&amp;E Mini DisplayPort to HDMI Female Adapter Cable for Apple Macbook, Macbook Pro
C&amp;E CNE86960 HDE HDMI Splitter Amplifier 1 In to 2 Out Dual Display
Connectland USB 2.0 External Enclosure for 3.5-Inch SATA/IDE Hard Drive CL-ENC35008
Metra 99-6510 Chry/Dodge/Jeep with NAV 04-UP Dash Kit
Swann 4-Channel DVR4-2550 and 4x ADS-180 CMOS SWDVK-425504C Cameras
Securifi Almond - (3 Minute Setup) Touchscreen Wireless Router / Range Extender
VideoSecu 5 Black Deluxe Speaker Mount Brackets for Walls and Ceilings 1XZ
Furman M-8X2 Merit Series 8 Outlet Power Conditioner and Surge Protector
nan
Amico DC 12V Open Frame Type Solenoid for Electric Door Lock
Portt